#INIT

In [ ]:
!pip install "dask[complete]"
!pip install pyarrow pandas
!pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.8/242.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from pathlib import Path
import os
import datetime
from datetime import timedelta
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import dask.dataframe as dd
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

#Load Data

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
# Read the parquet file using Dask
ddf = dd.read_parquet('/content/drive/MyDrive/merged_data.parquet')
df = ddf.compute()
df.head()

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,1655bf1b772d90df34c51ba9240f683d3847700b330295...,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391980769,VND,-650.0,-650.0,110000,...,<NA>,N,1|2,e4180c1cc088ebea7f7a882144f55159a8dddad215334f...,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
1,1655bf1b772d90df34c51ba9240f683d3847700b330295...,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391467506,VND,-34.0,-34.0,110000,...,<NA>,N,1|2,b403758c4a7b9fbc934464efaaaab53c7e24f0b68891f7...,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
2,1b781f92156f274627a131e707e7a890548aea921b84f5...,CASA,df774ebe64129d3a6cf3af75e7de8d01ca85e92b2f2c1c...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391730083,VND,120000.0,120000.0,110000,...,<NA>,Y,1|1,22527ba60412b7170a3dfcd831805f966b244b354d1607...,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-03
3,53048c1afff7f2cff6e1469fd1659f99ec93696df7a3ce...,CASA,2715ec2c8fc66cc32a6accc328d63843fb0dcd5565cd59...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391677652,VND,-62.0,-62.0,110000,...,<NA>,N,1|2,fb05380821892df14964a30cbe3fd795e0cb225c8da13b...,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
4,5abad7746ef69e998247ba7848ca6224dd106e30018237...,CASA,a5dc6ed17504c973d61d1ea548eb1945ae698847f3cb5c...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391519660,VND,-1500.0,-1500.0,110000,...,<NA>,N,1|2,54ff7b7a4f04b0a6e7de5abd7a4473ec7831e5579431a3...,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03


In [ ]:
# from merged data select the data where 'VALUEDATE'>= '2023-12-03'& 'VALUEDATE' <= '2023-12-07'

filtered_df = df[(df['VALUEDATE'] >= '2023-12-03') & (df['VALUEDATE'] <= '2023-12-07')]
filtered_df


,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391980769,VND,-650.0,-650.0,110000,...,<NA>,N,1|2,e4180c1cc088ebea7f7a882144f55159a8dddad215334fec1f741f658bfcbc1a,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
1,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391467506,VND,-34.0,-34.0,110000,...,<NA>,N,1|2,b403758c4a7b9fbc934464efaaaab53c7e24f0b68891f78329e964152de23684,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
2,1b781f92156f274627a131e707e7a890548aea921b84f5ddec59322ebc61e86d,CASA,df774ebe64129d3a6cf3af75e7de8d01ca85e92b2f2c1c44216c782d07183742,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391730083,VND,120000.0,120000.0,110000,...,<NA>,Y,1|1,22527ba60412b7170a3dfcd831805f966b244b354d1607a33d6b0efdeb592bac,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-03
3,53048c1afff7f2cff6e1469fd1659f99ec93696df7a3ce309aef278bfed89856,CASA,2715ec2c8fc66cc32a6accc328d63843fb0dcd5565cd590da6421ddfd950b810,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391677652,VND,-62.0,-62.0,110000,...,<NA>,N,1|2,fb05380821892df14964a30cbe3fd795e0cb225c8da13b3385ddb7b375241f15,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
4,5abad7746ef69e998247ba7848ca6224dd106e300182370090564f0cae691746,CASA,a5dc6ed17504c973d61d1ea548eb1945ae698847f3cb5c9fe44c5388cf45b807,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391519660,VND,-1500.0,-1500.0,110000,...,<NA>,N,1|2,54ff7b7a4f04b0a6e7de5abd7a4473ec7831e5579431a35bd21fceddaad0a1fb,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855088,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794685820,VND,4750.0,4750.0,110000,...,<NA>,N,1|2,98be8d28c01235ba77ff244ca6035dd86086b1ca28ca801a4fca515f57b3b9d6,MOB,HOI SO SHB,A2,VU THI YEN,NaN,2023-12-07
855089,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794734517,VND,-3500.0,-3500.0,110000,...,<NA>,N,1|2,4dc1194ff047ca5307f6848f4b69aa562c00e6022e5413205404eb622c803a6a,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-07
855090,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794734516,VND,900.0,900.0,110000,...,<NA>,Y,1|1,983fc2ba457d77883ea2a5160e77b81ffb98dc6e325abec2907a4ed8e43ec90c,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-07
855091,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794734518,VND,-850.0,-850.0,110000,...,<NA>,N,1|2,38213e45e4a821d58cc7f7ca3294ba14479168f67bf22ea53ca5a15e2eb8bc60,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-07


#Preprocess Data

In [ ]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4000821 entries, 0 to 855092
Data columns (total 26 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   CUST_CUSTNO            string        
 1   ACC_BUSINESSTYPE       string        
 2   ACC_ACCNO              string        
 3   ACC_BUSINESSNO         string        
 4   ACC_CURRENCYISO        string        
 5   BUSINESSNO_TRANS       string        
 6   FK_CURRENCY            string        
 7   AMOUNT                 float64       
 8   AMOUNTORIG             float64       
 9   BRANCH_OFFICE          string        
 10  CONTRA_ACCNO           string        
 11  CONTRA_ZIP             string        
 12  CONTRA_NAME            string        
 13  CSHYN                  string        
 14  REASON1                string        
 15  REASON2                string        
 16  REASON4                string        
 17  TR_FLAG_01             string        
 18  ANALYTICAL_TRANS_CODE  strin

In [ ]:
for column in filtered_df.columns:
    if pd.api.types.is_string_dtype(filtered_df[column]):
       filtered_df[column] = filtered_df[column].fillna('')
    elif pd.api.types.is_numeric_dtype(filtered_df[column]):
       filtered_df[column] = filtered_df[column].fillna(0)

In [ ]:
missing_values = filtered_df.isnull().sum()
missing_values

CUST_CUSTNO              0
ACC_BUSINESSTYPE         0
ACC_ACCNO                0
ACC_BUSINESSNO           0
ACC_CURRENCYISO          0
BUSINESSNO_TRANS         0
FK_CURRENCY              0
AMOUNT                   0
AMOUNTORIG               0
BRANCH_OFFICE            0
CONTRA_ACCNO             0
CONTRA_ZIP               0
CONTRA_NAME              0
CSHYN                    0
REASON1                  0
REASON2                  0
REASON4                  0
TR_FLAG_01               0
ANALYTICAL_TRANS_CODE    0
TR_SP_01                 0
TR_SP_02                 0
TR_SP_03                 0
TR_SP_04                 0
TR_SP_05                 0
TR_SP_10                 0
VALUEDATE                0
dtype: int64

In [ ]:

# Select only the necessary columns
filtered_df = filtered_df[['CUST_CUSTNO', 'AMOUNT']]

In [ ]:
filtered_df

,CUST_CUSTNO,AMOUNT
0,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,-650.0
1,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,-34.0
2,1b781f92156f274627a131e707e7a890548aea921b84f5ddec59322ebc61e86d,120000.0
3,53048c1afff7f2cff6e1469fd1659f99ec93696df7a3ce309aef278bfed89856,-62.0
4,5abad7746ef69e998247ba7848ca6224dd106e300182370090564f0cae691746,-1500.0
...,...,...
855088,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,4750.0
855089,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,-3500.0
855090,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,900.0
855091,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,-850.0


In [ ]:
# Group by customer number and aggregate the amounts into a list
transactions = filtered_df.groupby('CUST_CUSTNO')['AMOUNT'].apply(list).reset_index()

In [ ]:
transactions

,CUST_CUSTNO,AMOUNT
0,00001fd6f1d8852373810b5402eb3ae6adbd610d9d3608adb921498260fbc993,"[4800.0, -2000.0, -2800.0]"
1,0000700d5e830cfa894850d424f1713b9290eb5dfd6be661a354e70aab3d0f93,"[-400.0, -800.0, 400.0, -9.9, 10000.0, -26275.0, -24000.0, 24000.0, 5000.0, -575.0, 27000.0, -5000.0, -27000.0, 20000.0, 2.0, 2.0, -1000.0, -140.0, -179.8, 1000.0, -1000.0, 140.0, 1000.0, -50000.0, 10000.0, 37000.0, -850.0, 3000.0, -1700.0, -3000.0, 3000.0]"
2,0000c38422e04ccb772df72ed8ddac2b100b3bc88600be63ec3160d855dc3b6a,[150.0]
3,00010f06847bee388c69aaabcef8a3bc6221db4f9e03bfd34eec5e529e30b576,"[200.0, -800.0]"
4,0001271bf601ae6867e3f66c30e4a91c80dc0e2151cb58aa2744270a1f8fa135,"[3585.19, -700.0]"
...,...,...
334539,ffff97689f874c3808a5d80070e440de21eab1a16e133b7143cab3149c59b7c1,"[-400.0, -44.0]"
334540,ffff9cf7ba8d840b2a764286c65891355189bc72e33274501de1c932af7aa44e,"[3000.0, -10000.0, -3000.0, 10000.0]"
334541,ffffa7bf09b73311ed16c2d2fc2f27d4c6b09bd406d0a570046851ca9f2fecfb,"[-165000.0, 5000.0, 165000.0]"
334542,ffffaa126887038ed403e49535cebc14cc4f1c1a28e62cb67d99a884aaac3fb1,"[440.0, -440.0, -1800.0, 5000.0, 1800.0, -350.0, -5000.0, 500.0, 2750.0, 5000.0, -2750.0, -50.0, -170.0, 2200.0, -2200.0]"


In [ ]:
# Transform the data using TransactionEncoder
te = TransactionEncoder()
# First, converting each transaction amount to a string and wrapping it in a list
te_ary = te.fit(transactions['AMOUNT'].apply(lambda x: [str(i) for i in x])).transform(transactions['AMOUNT'].apply(lambda x: [str(i) for i in x]))


**This encoder is used to transform lists of transactions into a one-hot encoded format, which is suitable for association rule mining algorithms like FP-Growth.**

In [ ]:
df_tf = pd.DataFrame(te_ary, columns=te.columns_)# Converting the transformed array into a DataFrame with the appropriate column names

In [ ]:
df_tf

,-0.01,-0.02,-0.03,-0.04,-0.05,-0.06,-0.07,-0.08,-0.09,-0.1,...,99994.5,999949.15,99995.0,99995.89,99999.0,99999.93,99999.94,99999.99,9999990.47,9999996.0
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334539,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
334540,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
334541,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
334542,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


#Apply FP_Growth Mining

In [ ]:
# Apply FP-Growth
frequent_itemsets = fpgrowth(df_tf, min_support=0.01, use_colnames=True)

In [ ]:
frequent_itemsets

,support,itemsets
0,0.094331,(-2000.0)
1,0.134933,(-1000.0)
2,0.102100,(1000.0)
3,0.066189,(-3000.0)
4,0.059768,(-5000.0)
...,...,...
298,0.010997,"(-30.0, -10.0)"
299,0.013215,"(-1000.0, -1500.0)"
300,0.012507,"(-1500.0, -500.0)"
301,0.010241,"(-2000.0, -1500.0)"


In [ ]:
# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

In [ ]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(-2000.0, 1000.0)",(-1000.0),0.018362,0.134933,0.010151,0.552824,4.097031,0.007673,1.934513,0.770061
1,"(1000.0, -500.0)",(-1000.0),0.027691,0.134933,0.014249,0.514573,3.813543,0.010513,1.782073,0.758789
2,"(200.0, -100.0)",(-200.0),0.023844,0.114741,0.012330,0.517112,4.506763,0.009594,1.833258,0.797118
3,"(-200.0, -50.0)",(-100.0),0.026367,0.137049,0.014145,0.536447,3.914266,0.010531,1.861602,0.764687
4,"(100.0, -50.0)",(-100.0),0.018039,0.137049,0.010519,0.583099,4.254665,0.008046,2.069916,0.779017
5,"(-200.0, 100.0)",(-100.0),0.017926,0.137049,0.010564,0.589295,4.299875,0.008107,2.101143,0.781443
6,"(-200.0, 500.0)",(-500.0),0.023647,0.134242,0.011957,0.505625,3.766507,0.008782,1.751217,0.752292
7,"(-1000.0, 500.0)",(-500.0),0.023402,0.134242,0.012393,0.529570,3.944875,0.009251,1.840352,0.764395
8,(100000.0),(-100000.0),0.017773,0.020045,0.010172,0.572318,28.551356,0.009816,2.291314,0.982437
9,(-100000.0),(100000.0),0.020045,0.017773,0.010172,0.507456,28.551356,0.009816,1.994191,0.984714


**Normal Patterns:** Rules with high support and confidence values indicate frequent and consistent patterns in transactions. Customers associated with these patterns are likely normal.

**Anomalous Patterns:** Rules with lower support but high lift might indicate less frequent but significant patterns. High conviction and Zhang's metric values can also point to potential anomalies. Customers associated with these rules need further investigation.

In [ ]:
transactions_map = transactions[['CUST_CUSTNO', 'AMOUNT']]
transactions_map['AMOUNT'] = transactions_map['AMOUNT'].apply(lambda x: [str(i) for i in x])

In [ ]:
transactions_map

,CUST_CUSTNO,AMOUNT
0,00001fd6f1d8852373810b5402eb3ae6adbd610d9d3608adb921498260fbc993,"[4800.0, -2000.0, -2800.0]"
1,0000700d5e830cfa894850d424f1713b9290eb5dfd6be661a354e70aab3d0f93,"[-400.0, -800.0, 400.0, -9.9, 10000.0, -26275.0, -24000.0, 24000.0, 5000.0, -575.0, 27000.0, -5000.0, -27000.0, 20000.0, 2.0, 2.0, -1000.0, -140.0, -179.8, 1000.0, -1000.0, 140.0, 1000.0, -50000.0, 10000.0, 37000.0, -850.0, 3000.0, -1700.0, -3000.0, 3000.0]"
2,0000c38422e04ccb772df72ed8ddac2b100b3bc88600be63ec3160d855dc3b6a,[150.0]
3,00010f06847bee388c69aaabcef8a3bc6221db4f9e03bfd34eec5e529e30b576,"[200.0, -800.0]"
4,0001271bf601ae6867e3f66c30e4a91c80dc0e2151cb58aa2744270a1f8fa135,"[3585.19, -700.0]"
...,...,...
334539,ffff97689f874c3808a5d80070e440de21eab1a16e133b7143cab3149c59b7c1,"[-400.0, -44.0]"
334540,ffff9cf7ba8d840b2a764286c65891355189bc72e33274501de1c932af7aa44e,"[3000.0, -10000.0, -3000.0, 10000.0]"
334541,ffffa7bf09b73311ed16c2d2fc2f27d4c6b09bd406d0a570046851ca9f2fecfb,"[-165000.0, 5000.0, 165000.0]"
334542,ffffaa126887038ed403e49535cebc14cc4f1c1a28e62cb67d99a884aaac3fb1,"[440.0, -440.0, -1800.0, 5000.0, 1800.0, -350.0, -5000.0, 500.0, 2750.0, 5000.0, -2750.0, -50.0, -170.0, 2200.0, -2200.0]"


In [ ]:
te = TransactionEncoder()
te_ary = te.fit(transactions_map['AMOUNT']).transform(transactions_map['AMOUNT'])
df_tf_map = pd.DataFrame(te_ary, columns=te.columns_)
df_tf_map['CUST_CUSTNO'] = transactions_map['CUST_CUSTNO']

In [ ]:
df_tf_map

,-0.01,-0.02,-0.03,-0.04,-0.05,-0.06,-0.07,-0.08,-0.09,-0.1,...,999949.15,99995.0,99995.89,99999.0,99999.93,99999.94,99999.99,9999990.47,9999996.0,CUST_CUSTNO
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,00001fd6f1d8852373810b5402eb3ae6adbd610d9d3608adb921498260fbc993
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0000700d5e830cfa894850d424f1713b9290eb5dfd6be661a354e70aab3d0f93
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0000c38422e04ccb772df72ed8ddac2b100b3bc88600be63ec3160d855dc3b6a
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,00010f06847bee388c69aaabcef8a3bc6221db4f9e03bfd34eec5e529e30b576
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0001271bf601ae6867e3f66c30e4a91c80dc0e2151cb58aa2744270a1f8fa135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334539,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,ffff97689f874c3808a5d80070e440de21eab1a16e133b7143cab3149c59b7c1
334540,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,ffff9cf7ba8d840b2a764286c65891355189bc72e33274501de1c932af7aa44e
334541,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,ffffa7bf09b73311ed16c2d2fc2f27d4c6b09bd406d0a570046851ca9f2fecfb
334542,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,ffffaa126887038ed403e49535cebc14cc4f1c1a28e62cb67d99a884aaac3fb1


#Extract Customers from Classified Rules

In [ ]:
# Define thresholds
support_threshold = 0.01
lift_threshold = 4.0

# Classify rules
normal_rules = rules[(rules['support'] >= support_threshold) & (rules['lift'] < lift_threshold)]
anomalous_rules = rules[(rules['support'] >= support_threshold) & (rules['lift'] >= lift_threshold)]

In [ ]:
normal_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1,"(1000.0, -500.0)",(-1000.0),0.027691,0.134933,0.014249,0.514573,3.813543,0.010513,1.782073,0.758789
3,"(-200.0, -50.0)",(-100.0),0.026367,0.137049,0.014145,0.536447,3.914266,0.010531,1.861602,0.764687
6,"(-200.0, 500.0)",(-500.0),0.023647,0.134242,0.011957,0.505625,3.766507,0.008782,1.751217,0.752292
7,"(-1000.0, 500.0)",(-500.0),0.023402,0.134242,0.012393,0.529570,3.944875,0.009251,1.840352,0.764395


In [ ]:
anomalous_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(-2000.0, 1000.0)",(-1000.0),0.018362,0.134933,0.010151,0.552824,4.097031,0.007673,1.934513,0.770061
2,"(200.0, -100.0)",(-200.0),0.023844,0.114741,0.012330,0.517112,4.506763,0.009594,1.833258,0.797118
4,"(100.0, -50.0)",(-100.0),0.018039,0.137049,0.010519,0.583099,4.254665,0.008046,2.069916,0.779017
5,"(-200.0, 100.0)",(-100.0),0.017926,0.137049,0.010564,0.589295,4.299875,0.008107,2.101143,0.781443
8,(100000.0),(-100000.0),0.017773,0.020045,0.010172,0.572318,28.551356,0.009816,2.291314,0.982437
9,(-100000.0),(100000.0),0.020045,0.017773,0.010172,0.507456,28.551356,0.009816,1.994191,0.984714


In [ ]:
# Function to find matching transactions for a rule considering both antecedents and consequents
def find_matching_transactions(rule_antecedents, rule_consequents):
    mask_antecedents = df_tf_map[list(rule_antecedents)].all(axis=1)
    mask_consequents = df_tf_map[list(rule_consequents)].all(axis=1)
    mask = mask_antecedents & mask_consequents
    return df_tf_map[mask]

# Function to get matching customer numbers for rules
def get_matching_customers(rules):
    matching_transactions_list = []
    for index, rule in rules.iterrows():
        antecedents = rule['antecedents']
        consequents = rule['consequents']
        matching_transactions = find_matching_transactions(antecedents, consequents)
        matching_transactions_list.append(matching_transactions)
    matching_transactions = pd.concat(matching_transactions_list)
    matching_transactions = matching_transactions.drop_duplicates()
    matching_accounts = matching_transactions['CUST_CUSTNO'].unique().tolist()
    return matching_accounts

In [ ]:
# Get matching customers for normal and anomalous rules
normal_customers = get_matching_customers(normal_rules)

In [ ]:
anomalous_customers = get_matching_customers(anomalous_rules)

In [ ]:
# Print the results
print(f"Normal Customers: ")
normal_customers

Normal Customers: 


['...',
 ...]

In [ ]:
print(f"Anomalous Customers: ")
anomalous_customers

Anomalous Customers: 


['...',
 ...]

#Use other rules to find intersections

In [ ]:
filtered_df = df[(df['VALUEDATE'] >= '2023-12-03') & (df['VALUEDATE'] <= '2023-12-07')]
# Define the conditions for high-value withdrawals and deposits
withdrawal_condition = (filtered_df['AMOUNT'] <= -8000000) & (filtered_df['CSHYN'] == 'Y')
deposit_condition = (filtered_df['AMOUNT'] >= 6000000) & (filtered_df['CSHYN'] == 'Y')



In [ ]:
# Find customer numbers that match the conditions
high_value_withdrawals = filtered_df[withdrawal_condition]['CUST_CUSTNO'].unique().tolist()
high_value_deposits = filtered_df[deposit_condition]['CUST_CUSTNO'].unique().tolist()

In [ ]:

# Print the results
print("Customers with high-value withdrawals:")
high_value_withdrawals


Customers with high-value withdrawals:


['dd0cec29be4aed958e0e63310596bf67585cab965f0c5c5605adf03d7842e8dc',
 'c98591b2b6649b82dbcdffdc9fc034086e4f48a67f0a21f63d77a02292508ce7',
 '7e004469d2dbfc2b3d20b18f5e14e6297dc602ae6214514185f1ca3498300183',
 'b4834d8fda3438bb87e9bb4362e0820763e3c29d3e7a32b154156cb782392d72',
 'e75fd3752be01235f995711850615438995c70207e01362cc0b87c24276161bc',
 '7af6d692c1b382f16bb13079687250a8e6a4980fb02b340c134c89526bd1640f',
 '1d5b3fe3bd722434b3416a1be685f24643f8d0b4739aa0aa55fcb1d9285b656b',
 '0b5023c23745b483f339dcb5e74edca09fedbe5277ff09a1088406320d03f83b',
 '9dd602de6ba9a347939655bcfec434db7f2ddc7f81cca5525ae3d214e7a4418d',
 '037c993f5c86c4c64d4a1ad85d316930dfac2e1d2cf6a30079b924a28a8504ec',
 '860fc4310fbc0e7789500b2b185b1ddb3dcbdc8bc774ded4a256459281d47184',
 'c4da134db7d6011bd3b19155c9891be357ccb01ae646c99983b7f9eb557a1cd6',
 '8801d54648a8d67371b6ff5b59c301a6f4bb8ada611c740db39829ec722ba1ac',
 'effb5396524950318c196f1d2375a9345767f6e861321561b442d0af2ac74361',
 'f2544841ed8ec739b2a7e6695bae05c6

In [ ]:
print("Customers with high-value deposits:" )
high_value_deposits

Customers with high-value deposits:


['6d681af72190882763ad5115560d38bb219881aa0c68f6915bf54bc9a3d6c254',
 '0088f41fbbd53ea6d608e1cf2d959e32da7e6ef820e46d4b867fdb177119ae38',
 'dd0cec29be4aed958e0e63310596bf67585cab965f0c5c5605adf03d7842e8dc',
 'd69bc170f17102e4dbbfb9e371dfff5fce42235578ec226cface20a9de2a3d6a',
 '06a26e00a61fe8990647f04be42204c21cdb7b65aab487fb0ed0329644218ac5',
 '246c126837ec47030bdb208368be47a2be71f03b72a330cc44a4ed51d7cc4b47',
 '2d8bd3cbcfa3f19af52d5c3d8a8f7c79da042e5c90dd54f605c50dbfadd353c0',
 'ba9d3e403f87dbb3ad020acd9cbebb7c29c16001ceb8ab0ba5276bfdd5a7c265',
 '7e7d1b02f7e56085586c32b9466d5748046c2a7a78fdeda73d005ce5621e3e3f',
 '1247ebb1dd424b1c1eb89ad6c1ddc6e751d7f0968bb07110f8c37e45c1c250a0',
 'ce8b8613409b468031b4326a3ce62302458d4d515a57017cc73570243a0323c8',
 '27af8acbd2e904032dfae0aeefff3e6d85ea84e6e1ecebc5c5cc206f4b88fb40',
 'fce8e62518816f79636c8d62ac484d60b6b3f9dde75838f90a1e1e16d5708fd1',
 '419ee16675991ae260063c973874afcc09deb0ec70f3e1101d26fec9da2f8b1f',
 '1d5b3fe3bd722434b3416a1be685f246

In [ ]:
# Filter transactions that deposits or withdrawals of 100,000 VND often lead to an equivalent high-value transaction in the opposite direction
filtered_transactions = filtered_df[filtered_df['AMOUNT'].isin([100000, -100000])]
filtered_transactions


,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
2629,5404a9062c9d0f1b71254ada16c2aed7a80d8aacc70bed...,CASA,4e15c199b62d45f95ca3e01d433d8d333a10dace98af3e...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391550215,VND,-100000.0,-100000.0,110000,...,<NA>,N,1|2,b686b84e94e1bd1fae82ee39034a39182b41e3239c6acc...,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
4799,cec71af75236b7473bb855d77e9d569ede60e9ed0a30d3...,CASA,3cc067e762b50db7d3974bab6965427bd6c7f532d886a6...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391432404,VND,-100000.0,-100000.0,110000,...,<NA>,N,1|1,6fd5da89e63e9a0ca3b131019f4b1c88c257f22ec3edfa...,MOB,HOI SO SHB,A2,PHAN BOI NGHI,NaN,2023-12-03
5148,7708a7452244fdfaacea3af4b7c486c2a34c92bce318d1...,CASA,2d00887e340870c2188feb0a7c27cb5ae6fb9c90ac6a90...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391528979,VND,-100000.0,-100000.0,110000,...,<NA>,N,1|2,c17d8a723c96805d6b42ed2068dddd148a965b9cea1914...,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
5396,843016c5bbada94d695e189d4bbc3ab6c9536795bbce36...,CASA,879b5dc6df96969a341dd968fe28728e4ca113014c8b60...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391565990,VND,100000.0,100000.0,110000,...,u XXXXXXXXXXXXXXXX TRAN THI,Y,1|1,bbda665ad2212f5853553bf7e66d715e0ac7ef6e8b86a0...,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-03
6810,7d692fb126dbdff1727c59e21a30382f99e114f3c95578...,CASA,50e1af634f54c344e664c2df856df72b21a10659df7d75...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391567926,VND,-100000.0,-100000.0,110000,...,<NA>,N,1|2,69364012b6b05ee940626337bcbb254b433cabbdd789b7...,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852359,6e4da7f5cb88c1c5e6a219302ad63fef42d05ecb8191b4...,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,<NA>,2023120794694337,VND,100000.0,100000.0,110000,...,<NA>,N,21|45,b236827bd564d767300996741f45703500f40e2cc51b3d...,STP,HOI SO SHB,LP,CITAD SHB,NaN,2023-12-07
853749,b508057f87ec3d1f858541e8d6bee7aac50658e3f19778...,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,<NA>,2023120794808708,VND,-100000.0,-100000.0,110000,...,<NA>,N,1|2,387bedb762cf616b903a608f67b5134c2be94f71b3bcc1...,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-07
853791,6c85f1ffae2ce5b6f39be6625fddc80129cb2624665b96...,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,<NA>,2023120794882850,VND,-100000.0,-100000.0,110308,...,<NA>,N,2|1,439a2865188ad1c2eda2877a60e498c5c02f42046bbec5...,BRC,PGD SO 1/CN QUANG NINH,TD,<NA>,NaN,2023-12-07
854069,2fd6d688bded2e161e7674e48c8412c7729897ec371525...,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,<NA>,2023120795365478,VND,-100000.0,-100000.0,110000,...,<NA>,Y,1|2,a6adc51e077f77aecf1c8c7aeff9638d15957be340556d...,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-07


In [ ]:

# Group transactions by customer and count occurrences of 100,000 and -100,000
transaction_counts = filtered_transactions.groupby('CUST_CUSTNO')['AMOUNT'].value_counts().unstack(fill_value=0)

# Select customers with both deposits and withdrawals of 100,000
selected_customers = transaction_counts[(transaction_counts[100000] > 0) & (transaction_counts[-100000] > 0)]

# Print the selected customer numbers
selected_customers.index.tolist()


['...',
 ...]

In [ ]:
selected_customers

AMOUNT,-100000.0,100000.0
CUST_CUSTNO,,
0007c2cfad20bdb52d8285196af079100b824413fdbe41d4f14dedb85c9c5309,2,2
000975b0ee367856502b174994023821120d5730b3fcd384eda36753a7e1e2c7,2,1
000c7a8dc930b1c38d5283351cc9f8ed116b21678639f30ef643cbffb3fe594a,2,1
001921dda5635e50b4f619afc17fbb776adb5556b1ab08e09203048f64843ea9,1,1
0021e74dd8107882257e3587eea5b96b551b1edd3002b3c4d047b6df4c3a3fa2,2,1
...,...,...
ffb0b16a94c4f6ae1fedfde60bb7d2ec8eeeec35f2c1289bfe65229b6142f52e,1,1
ffbfb9dff789505b49ae502c7d095b631bc5f671dcb89b2d46c78bedb27cb257,1,1
fff19c110a51b1bc27ad3c1522cbca80d8841483dbeb87dc09c5b965e2563950,1,1


In [ ]:


# Create a DataFrame for selected customers and their amounts
selected_customer_df = filtered_transactions[filtered_transactions['CUST_CUSTNO'].isin(selected_customers.index.tolist())][['CUST_CUSTNO', 'AMOUNT','VALUEDATE','ACC_BUSINESSTYPE']]

# Display the DataFrame
selected_customer_df


,CUST_CUSTNO,AMOUNT,VALUEDATE,ACC_BUSINESSTYPE
7920,76d8b060fe13253fc93ae65072eca87fe32c3a46a34dc9...,-100000.0,2023-12-03,CASA
8304,76d8b060fe13253fc93ae65072eca87fe32c3a46a34dc9...,100000.0,2023-12-03,CASA
10445,fd8c6948039df262db8ff8594af572f11502dbef68419a...,-100000.0,2023-12-03,CASA
12486,f65f769b3e20dbe269cc8d5ac6c0195fadcd8cddfacc54...,100000.0,2023-12-03,CASA
12983,d51985753a0011e32a3b62e62a499a5720392d6fce1953...,100000.0,2023-12-03,CASA
...,...,...,...,...
849368,2fed8edb036a427d8a2b8409e2d4e50f79bbdcd4b6fa6c...,-100000.0,2023-12-07,<NA>
849390,2fed8edb036a427d8a2b8409e2d4e50f79bbdcd4b6fa6c...,100000.0,2023-12-07,<NA>
850420,02005478c6c830b712bd7ca87c82ceb743985f52d3ff40...,100000.0,2023-12-07,<NA>
850432,02005478c6c830b712bd7ca87c82ceb743985f52d3ff40...,-100000.0,2023-12-07,<NA>


In [ ]:

# Get the ACC_BUSINESSTYPE for the selected customers
selected_customer_data = filtered_transactions[filtered_transactions['CUST_CUSTNO'].isin(selected_customers.index.tolist())][['CUST_CUSTNO', 'ACC_BUSINESSTYPE']].drop_duplicates()

# Print the results
for _, row in selected_customer_data.iterrows():
  print(f"Customer {row['CUST_CUSTNO']}: ACC_BUSINESSTYPE = {row['ACC_BUSINESSTYPE']}")


Customer 76d8b060fe13253fc93ae65072eca87fe32c3a46a34dc9f897b9db42a46d2321: ACC_BUSINESSTYPE = CASA
Customer fd8c6948039df262db8ff8594af572f11502dbef68419a357199fffa0225a587: ACC_BUSINESSTYPE = CASA
Customer f65f769b3e20dbe269cc8d5ac6c0195fadcd8cddfacc54aa1198f6f8ae0b9b71: ACC_BUSINESSTYPE = CASA
Customer d51985753a0011e32a3b62e62a499a5720392d6fce195328fa813b2e320bf34c: ACC_BUSINESSTYPE = CASA
Customer d424102ef058feae86c255b1955e5ab3927bdb7ddab130013787ea46088fbfc0: ACC_BUSINESSTYPE = CASA
Customer 724b49e374f669b8067be3e9c2fc29e054f627198d30ac4cad1c94c14e172db2: ACC_BUSINESSTYPE = CASA
Customer e749e5d1380ee14ea496d23a715d62ae4ae3f14dd89b431b66f01b1a033a228d: ACC_BUSINESSTYPE = CASA
Customer df37fd74fe99503c667fbd1293e7631df14a7552efa097712d091a3df442284d: ACC_BUSINESSTYPE = CASA
Customer f78dfa08f03ef906f0d8030704259ea358ed78b5ea5a6612bba57c8d464cf06a: ACC_BUSINESSTYPE = CASA
Customer a3c6a598b49a25aa69fbaf72ec284813bd1b8835e3b9d39d5263669c86a2fb0d: ACC_BUSINESSTYPE = CASA
Customer b

In [ ]:

# Find intersection between high_value_withdrawals and selected customers
intersection_withdrawals_selected = set(high_value_withdrawals).intersection(set(selected_customers.index.tolist()))
print("Customers with both high-value withdrawals and frequent -100,000 VND transactions:")
intersection_withdrawals_selected


Customers with both high-value withdrawals and frequent -100,000 VND transactions:


{'0b5023c23745b483f339dcb5e74edca09fedbe5277ff09a1088406320d03f83b',
 '7dcc759f02fe2ab7ead08105ae5b8bc1e0ae1badb638151f035f8d080f3c4f6c',
 'be91ed7ac11d81ef2f701117c8148585fb59e88b2c4028974d52bf39440abee7'}

In [ ]:


# Iterate over the intersection and display corresponding transactions
for customer in intersection_withdrawals_selected:
  customer_transactions = selected_customer_df[selected_customer_df['CUST_CUSTNO'] == customer]
  print(f"Transactions for customer {customer}:")
  print(customer_transactions[['AMOUNT', 'VALUEDATE','ACC_BUSINESSTYPE']])
  print("\n")


Transactions for customer 0b5023c23745b483f339dcb5e74edca09fedbe5277ff09a1088406320d03f83b:
          AMOUNT  VALUEDATE ACC_BUSINESSTYPE
767157 -100000.0 2023-12-04             CASA
767394  100000.0 2023-12-04             CASA
282069  100000.0 2023-12-06             CASA


Transactions for customer 7dcc759f02fe2ab7ead08105ae5b8bc1e0ae1badb638151f035f8d080f3c4f6c:
         AMOUNT  VALUEDATE ACC_BUSINESSTYPE
11992 -100000.0 2023-12-06             TIDE
12238  100000.0 2023-12-06             TIDE


Transactions for customer be91ed7ac11d81ef2f701117c8148585fb59e88b2c4028974d52bf39440abee7:
          AMOUNT  VALUEDATE ACC_BUSINESSTYPE
696673 -100000.0 2023-12-04             CASA
168713 -100000.0 2023-12-04             CASA
168781  100000.0 2023-12-04             CASA
168995 -100000.0 2023-12-04             CASA
169217 -100000.0 2023-12-04             CASA
169971 -100000.0 2023-12-04             CASA
170015  100000.0 2023-12-04             CASA
170230 -100000.0 2023-12-04             CASA
170

In [ ]:


# Find intersection between high_value_deposits and selected customers
intersection_deposits_selected = set(high_value_deposits).intersection(set(selected_customers.index.tolist()))
print("Customers with both high-value deposits and frequent 100,000 VND transactions:")
intersection_deposits_selected

Customers with both high-value deposits and frequent 100,000 VND transactions:


{'0b5023c23745b483f339dcb5e74edca09fedbe5277ff09a1088406320d03f83b',
 '6d681af72190882763ad5115560d38bb219881aa0c68f6915bf54bc9a3d6c254',
 '7dcc759f02fe2ab7ead08105ae5b8bc1e0ae1badb638151f035f8d080f3c4f6c',
 'be91ed7ac11d81ef2f701117c8148585fb59e88b2c4028974d52bf39440abee7'}

In [ ]:

# Iterate over the intersection and display corresponding transactions
for customer in intersection_deposits_selected:
  customer_transactions = selected_customer_df[selected_customer_df['CUST_CUSTNO'] == customer]
  print(f"Transactions for customer {customer}:")
  print(customer_transactions[['AMOUNT', 'VALUEDATE','ACC_BUSINESSTYPE']])
  print("\n")


Transactions for customer 0b5023c23745b483f339dcb5e74edca09fedbe5277ff09a1088406320d03f83b:
          AMOUNT  VALUEDATE ACC_BUSINESSTYPE
767157 -100000.0 2023-12-04             CASA
767394  100000.0 2023-12-04             CASA
282069  100000.0 2023-12-06             CASA


Transactions for customer 6d681af72190882763ad5115560d38bb219881aa0c68f6915bf54bc9a3d6c254:
          AMOUNT  VALUEDATE ACC_BUSINESSTYPE
686939  100000.0 2023-12-04             CASA
694198  100000.0 2023-12-04             CASA
695811  100000.0 2023-12-04             CASA
886896 -100000.0 2023-12-04             CASA
887275 -100000.0 2023-12-04             CASA
...          ...        ...              ...
663255  100000.0 2023-12-07             CASA
664976 -100000.0 2023-12-07             CASA
665094 -100000.0 2023-12-07             CASA
665101 -100000.0 2023-12-07             CASA
797514  100000.0 2023-12-07             CASA

[87 rows x 3 columns]


Transactions for customer 7dcc759f02fe2ab7ead08105ae5b8bc1e0ae1badb63